In [1]:
import pandas as pd
df = pd.read_csv("../Files/Decision_Tree/titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
inputs = df.loc[:,['Pclass','Sex','Age','Fare']]
inputs

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500
...,...,...,...,...
886,2,male,27.0,13.0000
887,1,female,19.0,30.0000
888,3,female,NaN,23.4500
889,1,male,26.0,30.0000


In [3]:
from sklearn.preprocessing import LabelEncoder
len = LabelEncoder()

In [4]:
inputs.Sex = len.fit_transform(inputs.Sex)
inputs

,Pclass,Sex,Age,Fare
0,3,1,22.0,7.2500
1,1,0,38.0,71.2833
2,3,0,26.0,7.9250
3,1,0,35.0,53.1000
4,3,1,35.0,8.0500
...,...,...,...,...
886,2,1,27.0,13.0000
887,1,0,19.0,30.0000
888,3,0,NaN,23.4500
889,1,1,26.0,30.0000


In [5]:
rows_with_empty_values = inputs.isnull().any(axis=1)
rows_with_empty_values

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888     True
889    False
890    False
Length: 891, dtype: bool

In [6]:
inputs[rows_with_empty_values]

,Pclass,Sex,Age,Fare
5,3,1,NaN,8.4583
17,2,1,NaN,13.0000
19,3,0,NaN,7.2250
26,3,1,NaN,7.2250
28,3,0,NaN,7.8792
...,...,...,...,...
859,3,1,NaN,7.2292
863,3,0,NaN,69.5500
868,3,1,NaN,9.5000
878,3,1,NaN,7.8958


In [7]:
female = inputs[inputs['Sex']==0]
print('female mean age: ',female['Age'].mean(),' female median age: ',female['Age'].median())

female mean age:  27.915708812260537  female median age:  27.0


In [8]:
male = inputs[inputs['Sex']==1]
print('male mean age: ',male['Age'].mean(),' male median age: ',male['Age'].median())

male mean age:  30.72664459161148  male median age:  29.0


In [9]:
# inputs[(inputs['Sex']=='female') & (inputs['Age'].isnull())]['Age'].fillna(female['Age'].median(),inplace=True)
# inputs[(inputs['Sex']=='male') & (inputs['Age'].isnull())]['Age'].fillna(male['Age'].median(),inplace=True)
inputs['Age'] = inputs.apply(lambda row: female['Age'].median() if (row['Sex']==0 and pd.isnull(row['Age'])) else
                            (male['Age'].median() if (row['Sex']==1 and pd.isnull(row['Age'])) else row['Age']), axis=1)

In [10]:
current_empty_rows = inputs.isnull().any(axis='columns')
current_empty_rows

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Length: 891, dtype: bool

In [11]:
inputs[current_empty_rows]

,Pclass,Sex,Age,Fare


In [12]:
target = df.Survived

In [13]:
inputs

,Pclass,Sex,Age,Fare
0,3,1,22.0,7.2500
1,1,0,38.0,71.2833
2,3,0,26.0,7.9250
3,1,0,35.0,53.1000
4,3,1,35.0,8.0500
...,...,...,...,...
886,2,1,27.0,13.0000
887,1,0,19.0,30.0000
888,3,0,27.0,23.4500
889,1,1,26.0,30.0000


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)

In [15]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [16]:
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [17]:
model.score(X_test, y_test)

0.7486033519553073